# Testing environment for DoSE

## Setup

### Load libraries

In [1]:
import pandas as pd
import numpy as np 
import gseapy
from biothings_client import get_client

### Define data

In [2]:
seeds_file = "Input/0007079.txt"
betweenness_file = "Input/0007079_added_200_dmd_betweenness_hub_0.01.txt"
significance_file = "Input/0007079_added_200_dmd_significance_hub_1.txt"
diseases_file = "Input/ICD10_commROCG_raw.txt"
disease_clusters_file = "Input/ICD10_commROCG_cluster.txt"

### Load data

In [3]:
disease_id = "0007079"
seeds = pd.read_csv(seeds_file, sep="\t", header=None)[0]
betweenness = pd.read_csv(betweenness_file, sep="\t")['node']
significance = pd.read_csv(significance_file, sep="\t")['node']
diseases = pd.read_csv(diseases_file, sep="\t", header=None)
disease_clusters = pd.read_csv(disease_clusters_file, sep="\t", header=None)

In [4]:
import timeit
start = timeit.default_timer()

stop = timeit.default_timer()
print('Time: ', stop - start)  

Time:  4.699100099969655e-05


## Mapper

In [5]:
id_type_key = {'entrez':'entrezgene','ensembl':'ensembl.gene','symbol':'symbol','uniprot':'uniprot.Swiss-Prot','mondo':'mondo'}
gene_ids=['uniprot.Swiss-Prot','symbol','ensembl.gene','entrezgene']

In [22]:
def preprocess_results(mapping, multicol, singlecol, key, explode=False):
    
    def convert_to_string(cell, key):
        if str(cell) != 'nan':
            extracted_ids = [val.get(key) for val in cell]
            return ';'.join(str(e) for e in list(set(extracted_ids)))
        return cell
    mapping[multicol] = mapping[multicol].apply(lambda x: convert_to_string(x, key)) if multicol in mapping else np.nan
    if singlecol in mapping:
        mapping[multicol].fillna(mapping[singlecol], inplace=True)
        mapping = mapping.drop(columns=[singlecol])
    if explode:
        mapping = mapping[multicol].split(';').explode(multicol)
        mapping.rename(columns={multicol: singlecol}, inplace = True)
    return mapping


def get_prev_mapping(in_set, id_type, file, sep):
    # ===== Get mapping from local mapping file =====
    mapping = pd.read_csv(file, sep=sep, header=0, dtype=str)
    if id_type == "ICD-10":
        mapping = split_and_expand_column(data=mapping, split_string=",", column_name="ICD-10")
    # ==== Map given disease set ====
    id_type = id_type_key[id_type] if id_type in id_type_key else id_type
    mapped_set = mapping[mapping[id_type].isin(in_set)]
    # ===== Get missing values =====
    missing = list(set(in_set) - set(mapping[id_type]))
    return mapped_set, missing, mapping
    

def get_gene_mapping(gene_set, id_type):
    """
    Simple converter.

    :param gene_set: Set of gene ids
    :return: Dataframe
    """
    # ===== Get mapping from previous mappings =====
    df, missing, prev_mapping = get_prev_mapping(in_set=gene_set, id_type=id_type, file='gene_id_mapping.csv', sep=",")
    # ===== Get mapping for missing values =====
    if len(missing) > 0:
        mg = get_client("gene")
        mapping = mg.querymany(missing, scopes=id_type_key[id_type], fields=','.join(gene_ids),
                     species='human', returnall=False, as_dataframe=True, df_index=False)
        mapping = mapping.drop(columns=[id_type_key[id_type]])
        mapping.rename(columns={'query': id_type_key[id_type]}, inplace = True)
        # ===== Split if there are multiple ensembl ids =====
        if 'ensembl' in mapping:
            mapping = preprocess_results(mapping=mapping, multicol='ensembl', singlecol='ensembl.gene', key='gene', explode=True)
        mapping = mapping.drop(columns=['_id','_score'])
        # ===== Add results from missing values =====
        pd.concat([prev_mapping,mapping]).to_csv('gene_id_mapping.csv', index=False)
        df = pd.concat([df, mapping]).reset_index(drop=True)
    return df

def get_gene_to_attributes(gene_set, id_type):
    """
    Simple converter.

    :param gene_set: Set of gene ids
    :return: Dataframe
    """
    # ===== Get gene ID mappings =====
    gene_mapping, _, _ = get_prev_mapping(in_set=gene_set, id_type=id_type, file='gene_id_mapping.csv', sep=",")
    df, missing, prev_mapping = get_prev_mapping(in_set=set(gene_mapping['entrezgene']), id_type='entrez', file='gene_att_mapping.csv', sep=",")
    if len(missing) > 0:
        mg = get_client("gene")
        gene_ids=['uniprot.Swiss-Prot','symbol','ensembl.gene','entrezgene']
        mapping = mg.querymany(missing, scopes=','.join(gene_ids),
                            fields='pathway.kegg.id,go.BP.id,go.CC.id,go.MF.id',
                            species='human', returnall=False, as_dataframe=True, df_index=False)
        mapping.rename(columns={'query': 'entrezgene'}, inplace = True)
        for column in ['go.BP','go.CC','go.MF','pathway.kegg']:
            mapping = preprocess_results(mapping=mapping, multicol=column, singlecol=column+'.id', key='id')
        mapping = mapping.drop(columns=['_id','_score'])
        # ===== Add results from missing values =====
        pd.concat([prev_mapping,mapping]).to_csv('gene_att_mapping.csv', index=False)        
        df = pd.concat([df, mapping]).reset_index(drop=True)
    # work with not unique values...
    mapping_subset = gene_mapping[['entrezgene', id_type_key[id_type]]].drop_duplicates()
    df = pd.merge(mapping_subset, df, on = ['entrezgene'], how = 'outer')
    df = df.drop(columns=['entrezgene'])
    df = df.fillna('').groupby([id_type_key[id_type]], as_index=False).agg({i:combine_rows for i in ['pathway.kegg','go.BP','go.CC','go.MF']})
    return df


def combine_rows(x):
    return set(filter(None,';'.join(x).split(';')))

In [15]:
start = timeit.default_timer()
reference_mapping = get_gene_mapping(seeds, 'uniprot')
target_mapping = get_gene_mapping(significance, 'uniprot')
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.0062344479956664145


In [16]:
reference_mapping

,entrezgene,ensembl.gene,symbol,uniprot.Swiss-Prot
191,125,ENSG00000196616,ADH1B,P00325
192,2555,ENSG00000151834,GABRA2,P47869
193,126,ENSG00000248144,ADH1C,P00326
194,3356,ENSG00000102468,HTR2A,P28223


In [17]:
target_mapping

,entrezgene,ensembl.gene,symbol,uniprot.Swiss-Prot
0,1394,ENSG00000120088,CRHR1,P34998
1,1394,ENSG00000276191,CRHR1,P34998
2,104909134,ENSG00000263715,LINC02210-CRHR1,P34998
3,104909134,ENSG00000278232,LINC02210-CRHR1,P34998
4,104909134,ENSG00000282456,LINC02210-CRHR1,P34998
...,...,...,...,...
212,1742,ENSG00000132535,DLG4,P78352
213,4684,ENSG00000149294,NCAM1,P13591
214,2778,ENSG00000087460,GNAS,O95467
215,2776,ENSG00000156052,GNAQ,P50148


In [23]:
start = timeit.default_timer()
reference_kegg_mapping = get_gene_to_attributes(seeds, 'uniprot')
target_kegg_mapping = get_gene_to_attributes(significance, 'uniprot')
stop = timeit.default_timer()
print('Time: ', stop - start)

Time:  0.024431840996840037


In [24]:
reference_kegg_mapping

,uniprot.Swiss-Prot,pathway.kegg,go.BP,go.CC,go.MF
0,P00325,"{hsa05204, hsa00982, hsa00350, hsa00830, hsa00...","{GO:0042572, GO:0001523, GO:0006069, GO:0042573}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004024, GO:0008270, GO:0004745}"
1,P00326,"{hsa05204, hsa00982, hsa00350, hsa00830, hsa00...","{GO:0042572, GO:0006069, GO:0042573}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004024, GO:0004022, GO:0008270, GO:0004745}"
2,P28223,"{hsa04080, hsa04020, hsa04540, hsa04750, hsa04...","{GO:0008219, GO:0014832, GO:0007187, GO:004814...","{GO:0099055, GO:0043198, GO:0005886, GO:009905...","{GO:0042802, GO:0001618, GO:0071886, GO:004487..."
3,P47869,"{hsa04080, hsa05033, hsa05032, hsa04723, hsa04...","{GO:0060078, GO:0034220, GO:1904862, GO:004239...","{GO:0045202, GO:0099060, GO:0034707, GO:000588...","{GO:0005237, GO:0008503, GO:0022851, GO:000489..."


In [21]:
target_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,O00459,"{GO:0043409, GO:0043551, GO:0014065, GO:004685...","{GO:0005942, GO:0005829, GO:0005634}","{GO:0046935, GO:0001784, GO:0046982, GO:000551...","{hsa04914, hsa04926, hsa04668, hsa05211, hsa04..."
1,O14492,"{GO:0050873, GO:0019222, GO:0035556, GO:003003...","{GO:0005737, GO:0005886, GO:0001726, GO:000588...","{GO:0042802, GO:0005515, GO:0042169, GO:003559...","{hsa04722, hsa04910}"
2,O14610,"{GO:0007602, GO:0007186}",{GO:0005834},"{GO:0031681, GO:0003924}","{hsa04724, hsa05200, hsa04713, hsa04371, hsa05..."
3,O14775,"{GO:0036367, GO:1990603, GO:1901386, GO:000721...","{GO:0005737, GO:0098793, GO:0005834, GO:000563...","{GO:0005096, GO:0030159, GO:0005515, GO:003168...","{hsa04724, hsa05200, hsa04713, hsa04371, hsa05..."
4,O14842,"{GO:0032691, GO:0030073, GO:0051928, GO:000718...","{GO:0005887, GO:0005886}","{GO:0008289, GO:0045125, GO:0004930}",{hsa04911}
...,...,...,...,...,...
195,Q9UN70,"{GO:0050808, GO:0016339, GO:0007155, GO:0007156}","{GO:0005887, GO:0016020}",{GO:0005509},{}
196,Q9UNN8,"{GO:0050819, GO:0007596}","{GO:0005886, GO:0009986, GO:0005925, GO:000557...","{GO:0005515, GO:0038023}",{hsa04610}
197,Q9UQC2,"{GO:0030316, GO:0008284, GO:0043306, GO:000716...","{GO:0005737, GO:0005886, GO:0005829}","{GO:0005547, GO:0043325, GO:0005515, GO:0005068}","{hsa04664, hsa04380, hsa05220, hsa04071, hsa04..."
198,Q9Y2G0,"{GO:0046854, GO:0072659}","{GO:0015629, GO:0005886, GO:0005829}",{GO:0005515},{}


In [172]:
full_ids_mapping = pd.read_csv("../new_disorders.map", sep="\t", dtype=str)
full_ids_mapping

,mondo,omim,snomedct,umls,orpha,mesh,ncit,doid,meddra,medgen,ICD-10
0,0008118,164330,716180009,C1834013,2724,C537740,NaN,NaN,NaN,NaN,NaN
1,0010439,300829,NaN,C1853577,NaN,C543241,NaN,NaN,NaN,NaN,NaN
2,0008117,164310,763829004,C1834014,98897,C563508,NaN,NaN,NaN,NaN,"G71,G71.0"
3,0009448,242600,84121007,C0268654,42062,C536285,NaN,NaN,NaN,NaN,"E72,E72.0"
4,0008119,164400,715748006,C0752120,98755,NaN,C129982,0050954,NaN,NaN,"G11,G11.8"
...,...,...,...,...,...,...,...,...,...,...,...
24115,0009507,245550,732961003,C1855551,1296,C538396,NaN,NaN,NaN,NaN,"Q87,Q87.8"
24116,0009508,245552,NaN,C1855550,NaN,C537549,NaN,NaN,NaN,NaN,NaN
24117,0009501,245340,766715000,C1855577,171690,C565449,NaN,NaN,NaN,NaN,"G72,G72.8"
24118,0009502,245348,NaN,C1855565,79244,C565448,NaN,NaN,NaN,NaN,"E74,E74.4"


In [14]:
full_ids_mapping.count()

mondo       24120
omim         8841
snomedct     8962
umls        16234
orpha        9363
mesh         8075
ncit         6953
doid         8944
meddra       1144
medgen          1
ICD-10       9561
dtype: int64

In [134]:
def split_and_expand_column(data, split_string, column_name):
    s = data[column_name].str.split(split_string, expand=True).stack()
    i = s.index.get_level_values(0)
    df2 = data.loc[i].copy()
    df2[column_name] = s.values
    return df2

def get_disease_mapping(disease_set, id_type):
    # ==== Get Mondo IDs ====
    disease_id_set,_,_ = get_prev_mapping(in_set=disease_set, id_type=id_type, file="../new_disorders.map", sep="\t")
    mondo_set = list(set('MONDO:'+disease_id_set['mondo']))
    # ===== Get mapping from previous mappings =====
    df, missing, prev_mapping = get_prev_mapping(in_set=mondo_set, id_type='mondo', file='disease_disgenet_mapping.csv', sep=",")
    # ==== Get disgenet values ====
    if len(missing) > 0:
        md = get_client("disease")
        mapping = md.getdiseases(missing,
                                 fields='disgenet.genes_related_to_disease.gene_id,disgenet.variants_related_to_disease.rsid,ctd.pathway_related_to_disease.kegg_pathway_id',
                                 species='human', returnall=False, as_dataframe=True, df_index=False)
        mapping.rename(columns={'query': 'mondo'}, inplace = True)
        # transform dataframe to combine single and multiple results
        mapping = preprocess_results(mapping=mapping, multicol='disgenet.genes_related_to_disease', 
                                     singlecol='disgenet.genes_related_to_disease.gene_id', key='gene_id')
        mapping = preprocess_results(mapping=mapping, multicol='disgenet.variants_related_to_disease', 
                                     singlecol='disgenet.variants_related_to_disease.rsid', key='rsid')
        mapping = preprocess_results(mapping=mapping, multicol='ctd.pathway_related_to_disease', 
                                     singlecol='ctd.pathway_related_to_disease.kegg_pathway_id', key='kegg_pathway_id')
        mapping = mapping.drop(columns=['_id','_version','disgenet._license']) 
        # ==== Get pathways from file ====
        mondo_to_pathway = pd.read_csv('mondo_to_pathways.csv')
        mapping = mapping.merge(mondo_to_pathway, on='mondo', how='left')
        #  work with nan float values
        mapping = mapping.fillna('')
        mapping = mapping.astype(str)
        # combine with ctd pathway mapping 
        mapping.loc[:,('ctd.pathway_related_to_disease')] = (mapping.loc[:,('ctd.pathway_related_to_disease')] + ";" + mapping.loc[:,('pathways')]).str.strip(';')
        mapping = mapping.drop(columns=['pathways'])
        mapping = mapping.drop_duplicates()
        # ===== Add results from missing values =====
        pd.concat([prev_mapping,mapping]).to_csv('disease_disgenet_mapping.csv', index=False)
        df = pd.concat([df, mapping]).reset_index(drop=True)
    # ==== Map back to previous ids ====
    df.loc[:,("mondo")] = df.loc[:,("mondo")].str.split(':').str[1]
    # work with not unique values...
    columns = ['mondo', id_type] if id_type != 'mondo' else ['mondo']
    mapping_subset = disease_id_set[columns].drop_duplicates()
    df = pd.merge(mapping_subset, df, on = ['mondo'], how = 'outer')
    df = df.drop(columns=['mondo']) if id_type != 'mondo' else df
    df = df.fillna('').groupby(id_type, as_index = False).agg({'disgenet.genes_related_to_disease': combine_rows, 'disgenet.variants_related_to_disease': combine_rows, 'ctd.pathway_related_to_disease': combine_rows})
    return df

In [87]:
diseases[0]

0     E10
1     E11
2     E12
3     E13
4     E14
5     E66
6     F00
7     F01
8     F02
9     F03
10    G20
11    G30
12    G43
13    I10
14    I11
15    I12
16    I13
17    I15
18    I21
19    I22
20    I50
21    I63
22    I64
23    I70
24    J45
Name: 0, dtype: object

In [135]:
start = timeit.default_timer()
disease_df = get_disease_mapping(disease_set=diseases[0], id_type='ICD-10')
stop = timeit.default_timer()
print('Time: ', stop - start)

querying 1-117...done.
Time:  3.0007637850067113


In [59]:
disease_df

,ICD-10,disgenet.genes_related_to_disease,disgenet.variants_related_to_disease,ctd.pathway_related_to_disease
0,E10,"{3845, 3485, 1314, 5126, 3642, 5913, 6513, 257...","{rs560887, rs222826, rs1864169, rs2890565, rs1...","{hsa04658, hsa05200, hsa04152, hsa00590, hsa00..."
1,E11,"{4825, 6927, 4760, 2348, 5335, 9451, 6513, 363...","{rs193922600, rs587776771, rs104894008, rs1555...","{hsa04658, hsa05200, hsa_M00341, hsa04152, hsa..."
2,E13,"{107075310, 6343, 3643, 9451, 150159, 7466, 23...","{rs1190356035, rs71524377, rs1801208, rs777580...","{hsa05010, hsa04520, hsa05164, hsa05022, hsa04..."
3,E14,"{3845, 3485, 1314, 5126, 3642, 5913, 6513, 257...","{rs560887, rs222826, rs1864169, rs2890565, rs1...","{hsa05200, hsa04152, hsa00590, hsa04930, hsa04..."
4,E66,"{84808, 3845, 27344, 9967, 278, 27342, 3485, 1...","{rs2067853, rs9356744, rs4714010, rs16147, rs2...","{hsa05200, hsa04152, hsa04020, hsa05207, hsa04..."
5,F00,"{5521, 6927, 27344, 10592, 65018, 105274375, 2...","{rs17075286, rs821616, rs9470080, rs993804, rs...",{}
6,F01,"{5444, 4318, 9104, 185, 5648, 9402, 551, 6197,...","{rs1555729604, rs699, rs1554952291, rs429358, ...","{hsa04658, hsa05200, hsa05165, hsa04330, hsa05..."
7,G20,"{5444, 84894, 4318, 1996, 6622.0, 65018, 12127...","{rs80356718, rs1272596579, rs71799110, rs10488...","{hsa04658, hsa05200, hsa00053, hsa04152, hsa_M..."
8,G30,"{2827, 84894, 126017, 27344, 27342, 3485, 1114...","{rs63750215, rs3740677, rs676134, rs61510607, ...","{hsa04658, hsa05200, hsa04152, hsa00330, hsa04..."
9,G43,"{1910, 773, 9839, 185, 63976, 348, 6648, 23063...","{rs228648, rs2890565, rs1801132, rs766474188, ...","{hsa05200, hsa05010, hsa04520, hsa04020, hsa04..."


In [29]:
def make_setup():
    def convert_to_string(elements):
        if str(elements) != 'nan':
            return ';'.join(str(e) for e in elements)
        return cell
    
    def get_df_from_url(content, column_names, header=None):
        df = pd.read_csv(io.StringIO(content), sep='\t', names=column_names, header=header, dtype=str)
        df.fillna('NULL', inplace=True)
        return df

    omim_to_hsa = get_df_from_url(
            content=io.TextIOWrapper(urlopen("http://rest.genome.jp/link/omim/hsa"), encoding="UTF-8").read(),
            column_names=['hsa', 'omim', 'reverse'])
    omim_to_hsa = omim_to_hsa[['hsa','omim']]
    hsa_to_pathway = get_df_from_url(
            content=io.TextIOWrapper(urlopen("http://rest.kegg.jp/link/pathway/hsa"), encoding="UTF-8").read(),
                                             column_names=['hsa', 'pathways'])
    hsa_to_pathway['pathways'] = hsa_to_pathway['pathways'].str.split(':').str[1]
    hsa_to_pathway = hsa_to_pathway.groupby('hsa', as_index = False).agg({'pathways': convert_to_string})

    omim_to_pathway = omim_to_hsa.merge(hsa_to_pathway, on='hsa', how='left')[['omim','pathways']]
    omim_to_pathway['omim'] = omim_to_pathway['omim'].str.split(':').str[1]
    
    full_ids_mapping = pd.read_csv("../new_disorders.map", sep="\t", dtype=str)
    mondo_to_pathway = full_ids_mapping[['mondo','omim']].merge(omim_to_pathway, on='omim')
    mondo_to_pathway['mondo'] = 'MONDO:'+mondo_to_pathway['mondo']
    
    mondo_to_pathway[['mondo','pathways']].to_csv('mondo_to_pathways.csv', index=False)
    return mondo_to_pathway

In [30]:
make_setup()

NameError: name 'io' is not defined

# do the comparisson now

### set to set

In [31]:
reference_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,P00325,"{GO:0042572, GO:0001523, GO:0042573, GO:0006069}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004745, GO:0004024, GO:0008270}","{hsa00350, hsa05204, hsa00980, hsa00071, hsa00..."
1,P00326,"{GO:0006069, GO:0042573, GO:0042572}","{GO:0005654, GO:0005886, GO:0005829}","{GO:0004745, GO:0004024, GO:0004022, GO:0008270}","{hsa00350, hsa05204, hsa00980, hsa00071, hsa00..."
2,P28223,"{GO:0045907, GO:0050965, GO:0050966, GO:000756...","{GO:0098978, GO:0031410, GO:0043025, GO:000582...","{GO:0071886, GO:0001618, GO:0001965, GO:000499...","{hsa04750, hsa04540, hsa04020, hsa04080, hsa04..."
3,P47869,"{GO:0042391, GO:0001505, GO:0007165, GO:005193...","{GO:0043025, GO:0098794, GO:0030424, GO:004300...","{GO:0004890, GO:0005237, GO:0008503, GO:003059...","{hsa04723, hsa04727, hsa05032, hsa04742, hsa04..."


In [32]:
target_kegg_mapping

,uniprot.Swiss-Prot,go.BP,go.CC,go.MF,pathway.kegg
0,O00459,"{GO:0051897, GO:0051056, GO:0015031, GO:004801...","{GO:0005942, GO:0005634, GO:0005829}","{GO:0046982, GO:0001784, GO:0046935, GO:001990...","{hsa05200, hsa04152, hsa04930, hsa04722, hsa04..."
1,O14492,"{GO:0007596, GO:0019222, GO:0035556, GO:005087...","{GO:0001726, GO:0001725, GO:0005829, GO:000588...","{GO:0042802, GO:0035591, GO:0042169, GO:000506...","{hsa04910, hsa04722}"
2,O14610,"{GO:0007186, GO:0007602}",{GO:0005834},"{GO:0003924, GO:0031681}","{hsa05200, hsa04728, hsa04723, hsa04727, hsa04..."
3,O14775,"{GO:0006457, GO:1901386, GO:0007165, GO:000718...","{GO:0005829, GO:0005634, GO:0005834, GO:009879...","{GO:0030159, GO:0005096, GO:0003924, GO:003168...","{hsa05200, hsa04728, hsa04723, hsa04727, hsa04..."
4,O14842,"{GO:0051928, GO:0030073, GO:0050796, GO:004259...","{GO:0005886, GO:0005887}","{GO:0004930, GO:0045125, GO:0008289}",{hsa04911}
...,...,...,...,...,...
195,Q9UN70,"{GO:0007155, GO:0050808, GO:0007156, GO:0016339}","{GO:0005887, GO:0016020}",{GO:0005509},{}
196,Q9UNN8,"{GO:0007596, GO:0050819}","{GO:0005925, GO:0005813, GO:0005615, GO:004847...","{GO:0038023, GO:0005515}",{hsa04610}
197,Q9UQC2,"{GO:0051897, GO:0038095, GO:0043306, GO:003031...","{GO:0005886, GO:0005829, GO:0005737}","{GO:0043325, GO:0005515, GO:0005547, GO:0005068}","{hsa04072, hsa04666, hsa05220, hsa04014, hsa04..."
198,Q9Y2G0,"{GO:0072659, GO:0046854}","{GO:0005886, GO:0015629, GO:0005829}",{GO:0005515},{}


In [199]:
def create_ref_dict(mapping, keys):
    reference_dict = dict()
    for att_type in keys:
        reference_dict[att_type] = set.union(*mapping[att_type])
    return reference_dict

In [200]:
def evaluate_values(mapping, ref_dict, threshold, keys):
    def get_intersection(values_set, ref_set):
        if len(values_set) == 0:
            return 0.0
        return (len(values_set & ref_set)/len(values_set))
    
    evaluation = list()
    for attribute in keys:
        evaluated_series = mapping[attribute].apply(get_intersection, ref_set=ref_dict[attribute])
        evaluation.append([attribute, str(len(evaluated_series[evaluated_series > threshold])/len(evaluated_series))])
    return evaluation

In [201]:
def compare_set_to_set(ref, ref_id_type, targets, targets_id_type, threshold=0.0):
    reference_mapping = get_gene_to_attributes(ref, ref_id_type)
    target_mapping = get_gene_to_attributes(targets, targets_id_type)
    ref_dict = create_ref_dict(mapping=reference_mapping, keys=reference_mapping.columns[1:])
    result = evaluate_values(mapping=target_mapping, ref_dict=ref_dict, threshold=threshold, keys=target_mapping.columns[1:])
    return result

In [202]:
compare_set_to_set(ref=seeds, ref_id_type='uniprot', targets=significance, targets_id_type='uniprot')

[['go.BP', '0.74'],
 ['go.CC', '0.91'],
 ['go.MF', '0.91'],
 ['pathway.kegg', '0.365']]

In [203]:
def compare_id_to_set(ref_id, ref_id_type, targets, targets_id_type, threshold=0.0):
    disease_id_atts = get_disease_mapping({ref_id}, ref_id_type)
    disease_id_atts = disease_id_atts.rename(columns={'ctd.pathway_related_to_disease': 'pathway.kegg'})
    target_mapping = get_gene_to_attributes(targets, targets_id_type)
    ref_dict = create_ref_dict(mapping=disease_id_atts, keys=['pathway.kegg'])
    result = evaluate_values(mapping=target_mapping, ref_dict=ref_dict, threshold=threshold, keys=['pathway.kegg'])
    return result

In [204]:
compare_id_to_set(ref_id=disease_id, ref_id_type='mondo', targets=significance, targets_id_type='uniprot')

/home/kikky/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


[['pathway.kegg', '0.365']]

In [91]:
overall_missing_values = len(diseases[0])-len(disease_df)
overall_missing_values

5

In [205]:
def get_distance_matrix(eval_set):
    dis_mat = np.zeros((len(eval_set), len(eval_set)))
    for index1 in range(0, len(eval_set)):
        for index2 in range(index1, len(eval_set)):
            calc_dis = 1-(len(eval_set[index1] & eval_set[index2]) / min(len(eval_set[index1]),len(eval_set[index2])))
            # assign to matrix
            dis_mat[index1][index2] = calc_dis
            dis_mat[index2][index1] = calc_dis
    return dis_mat

In [206]:
def compare_ids(disease_id_set, id_type):
    disease_df = get_disease_mapping(disease_set=disease_id_set, id_type=id_type)
    result = list()
    for attribute in disease_df.columns[1:]:
        subset_df = disease_df[disease_df[attribute].str.len() > 0].reset_index()
        missing_values = len(disease_id_set)-len(subset_df)
        print("Missing values for "+attribute+" :"+str(missing_values)+"/"+str(len(disease_id_set)))
        comp_mat = get_distance_matrix(eval_set = subset_df[attribute])
        comp_mean = (comp_mat.sum() - np.diag(comp_mat).sum())/(len(subset_df[attribute])*(len(subset_df[attribute])-1))
        result.append([attribute, comp_mean])
    return result

In [207]:
compare_ids(disease_id_set=diseases[0],id_type='ICD-10')

Missing values for disgenet.genes_related_to_disease :7/25
Missing values for disgenet.variants_related_to_disease :8/25
Missing values for ctd.pathway_related_to_disease :8/25


[['disgenet.genes_related_to_disease', 0.5526086117259117],
 ['disgenet.variants_related_to_disease', 0.9030081235039393],
 ['ctd.pathway_related_to_disease', 0.29216473924553843]]

In [208]:
disease_clusters

,0,1,2,cluster_index
0,G43,567,Migraine,11
1,I64,470,"Stroke, not specified as haemorrhage or infarc...",10
2,E10,438,Type1 diabetes mellitus,9
3,E11,438,Type2 diabetes mellitus,9
4,J45,438,Asthma,9
5,E13,438,Other specified diabetes mellitus,9
6,E66,438,Obesity,9
7,G30,438,Alzheimer,9
8,I15,438,Secondary hypertension,9
9,I21,438,Acute myocardial infarction,9


In [209]:
disease_clusters_df = get_disease_mapping(disease_set=disease_clusters[0], id_type='ICD-10')
disease_clusters_df

/home/kikky/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1951: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[selected_item_labels] = value


,ICD-10,disgenet.genes_related_to_disease,disgenet.variants_related_to_disease,ctd.pathway_related_to_disease
0,E10,"{3845, 3485, 1314, 5126, 3642, 5913, 6513, 257...","{rs560887, rs222826, rs1864169, rs1232898090, ...","{hsa04658, hsa05200, hsa04152, hsa00590, hsa00..."
1,E11,"{4825, 6927, 4760, 2348, 5335, 9451, 6513, 363...","{rs193922600, rs587776771, rs104894008, rs1555...","{hsa04658, hsa05200, hsa_M00341, hsa04152, hsa..."
2,E13,"{107075310, 6343, 3643, 9451, 150159, 7466, 23...","{rs1190356035, rs71524377, rs1801208, rs777580...","{hsa05010, hsa04520, hsa05164, hsa05022, hsa04..."
3,E14,"{3845, 3485, 1314, 5126, 3642, 5913, 6513, 257...","{rs560887, rs222826, rs1864169, rs1232898090, ...","{hsa05200, hsa04152, hsa00590, hsa04930, hsa04..."
4,E66,"{84808, 3845, 27344, 9967, 278, 27342, 3485, 1...","{rs2067853, rs9356744, rs2048417, rs16147, rs4...","{hsa05200, hsa04152, hsa04020, hsa05207, hsa04..."
5,F00,"{5521, 6927, 27344, 10592, 65018, 105274375, 2...","{rs17075286, rs821616, rs9470080, rs993804, rs...",{}
6,F01,"{5444, 4318, 9104, 185, 5648, 9402, 551, 6197,...","{rs1555729604, rs699, rs1554952291, rs429358, ...","{hsa04658, hsa05200, hsa05165, hsa04330, hsa01..."
7,G20,"{5444, 84894, 4318, 1996, 6622.0, 65018, 12127...","{rs80356718, rs1272596579, rs71799110, rs10488...","{hsa04658, hsa05200, hsa00053, hsa00590, hsa04..."
8,G30,"{2827, 84894, 126017, 27344, 27342, 3485, 1114...","{rs63750215, rs3740677, rs676134, rs61510607, ...","{hsa04658, hsa05200, hsa04152, hsa00330, hsa04..."
9,G43,"{1910, 773, 9839, 185, 63976, 348, 6648, 23063...","{rs228648, rs2890565, rs1801132, rs766474188, ...","{hsa05010, hsa05200, hsa04520, hsa04020, hsa04..."


In [252]:
subset_df = disease_clusters_df[disease_clusters_df['ctd.pathway_related_to_disease'].str.len() > 0].reset_index(drop=True)
subset_df

,ICD-10,disgenet.genes_related_to_disease,disgenet.variants_related_to_disease,ctd.pathway_related_to_disease
0,E10,"{3845, 3485, 1314, 5126, 3642, 5913, 6513, 257...","{rs560887, rs222826, rs1864169, rs1232898090, ...","{hsa04658, hsa05200, hsa04152, hsa00590, hsa00..."
1,E11,"{4825, 6927, 4760, 2348, 5335, 9451, 6513, 363...","{rs193922600, rs587776771, rs104894008, rs1555...","{hsa04658, hsa05200, hsa_M00341, hsa04152, hsa..."
2,E13,"{107075310, 6343, 3643, 9451, 150159, 7466, 23...","{rs1190356035, rs71524377, rs1801208, rs777580...","{hsa05010, hsa04520, hsa05164, hsa05022, hsa04..."
3,E14,"{3845, 3485, 1314, 5126, 3642, 5913, 6513, 257...","{rs560887, rs222826, rs1864169, rs1232898090, ...","{hsa05200, hsa04152, hsa00590, hsa04930, hsa04..."
4,E66,"{84808, 3845, 27344, 9967, 278, 27342, 3485, 1...","{rs2067853, rs9356744, rs2048417, rs16147, rs4...","{hsa05200, hsa04152, hsa04020, hsa05207, hsa04..."
5,F01,"{5444, 4318, 9104, 185, 5648, 9402, 551, 6197,...","{rs1555729604, rs699, rs1554952291, rs429358, ...","{hsa04658, hsa05200, hsa05165, hsa04330, hsa01..."
6,G20,"{5444, 84894, 4318, 1996, 6622.0, 65018, 12127...","{rs80356718, rs1272596579, rs71799110, rs10488...","{hsa04658, hsa05200, hsa00053, hsa00590, hsa04..."
7,G30,"{2827, 84894, 126017, 27344, 27342, 3485, 1114...","{rs63750215, rs3740677, rs676134, rs61510607, ...","{hsa04658, hsa05200, hsa04152, hsa00330, hsa04..."
8,G43,"{1910, 773, 9839, 185, 63976, 348, 6648, 23063...","{rs228648, rs2890565, rs1801132, rs766474188, ...","{hsa05010, hsa05200, hsa04520, hsa04020, hsa04..."
9,I12,"{28996, 1241, 4914, 81, 2255, 285, 7170, 80350...",{rs11739136},"{hsa04512, hsa05200, hsa00590, hsa05152, hsa04..."


In [253]:
dist_mat = get_distance_matrix(subset_df['ctd.pathway_related_to_disease'])
dist_df = pd.DataFrame(dist_mat, columns = subset_df['ICD-10'], index = subset_df['ICD-10'])
dist_df

ICD-10,E10,E11,E13,E14,E66,F01,G20,G30,G43,I12,I15,I21,I22,I50,I63,I70,J45
ICD-10,,,,,,,,,,,,,,,,,
E10,0.000000,0.092784,0.105263,0.000000,0.342105,0.454545,0.225131,0.139175,0.147059,0.092593,0.092784,0.157895,0.216495,0.139535,0.225806,0.186747,0.185567
E11,0.092784,0.000000,0.105263,0.074380,0.157895,0.090909,0.162304,0.105263,0.000000,0.000000,0.107884,0.000000,0.080000,0.023256,0.032258,0.078313,0.116505
E13,0.105263,0.105263,0.000000,0.263158,0.842105,1.000000,0.078947,0.131579,0.852941,0.815789,0.131579,0.894737,0.184211,0.868421,0.967742,0.263158,0.210526
E14,0.000000,0.074380,0.263158,0.000000,0.421053,0.818182,0.206612,0.107438,0.411765,0.425926,0.057851,0.368421,0.190083,0.232558,0.419355,0.314050,0.173554
E66,0.342105,0.157895,0.842105,0.421053,0.000000,0.909091,0.421053,0.342105,0.764706,0.868421,0.289474,0.894737,0.315789,0.736842,0.774194,0.394737,0.315789
F01,0.454545,0.090909,1.000000,0.818182,0.909091,0.000000,0.454545,0.181818,0.545455,0.727273,0.363636,0.727273,0.454545,0.636364,0.636364,0.363636,0.454545
G20,0.225131,0.162304,0.078947,0.206612,0.421053,0.454545,0.000000,0.193717,0.205882,0.222222,0.167539,0.157895,0.282723,0.255814,0.129032,0.240964,0.204188
G30,0.139175,0.105263,0.131579,0.107438,0.342105,0.181818,0.193717,0.000000,0.088235,0.074074,0.119617,0.052632,0.195000,0.116279,0.064516,0.162651,0.213592
G43,0.147059,0.000000,0.852941,0.411765,0.764706,0.545455,0.205882,0.088235,0.000000,0.382353,0.000000,0.736842,0.058824,0.647059,0.741935,0.088235,0.088235


## Statistics

In [323]:
def min_inter_distance_to_entity(entity, distance_matrix, current_cluster, ids_cluster):
    min_distance = None
    for cluster in ids_cluster['cluster_index'].unique():
        if cluster != current_cluster:
            cluster_member = ids_cluster[ids_cluster['cluster_index']==cluster][0]
            distance = inter_distance([entity], cluster_member, distance_matrix, 'average')
            if min_distance is None or min_distance > distance:
                min_distance = distance
    return min_distance


def intra_distance(cluster, distance_matrix, linkage, entity=None):
    distances = []
    if len(cluster) == 1:
        return 0
    for i in range(len(cluster)):
        if entity is not None:
            if cluster[i] != entity:
                if entity in distance_matrix.columns and cluster[i] in distance_matrix.columns:
                    distances.append(distance_matrix[entity][cluster[i]])
        else:
            for j in range(i+1,len(cluster)):
                if cluster[j] in distance_matrix.columns and cluster[i] in distance_matrix.columns:
                    distances.append(distance_matrix[cluster[j]][cluster[i]])
    calculations = len(cluster) * (len(cluster)-1) if entity is None else len(cluster)-1
    if linkage == 'average':
        return sum(distances)/calculations
    if linkage == 'complete':
        return max(distances)

def inter_distance(cluster_one, cluster_two, distance_matrix, linkage):
    distances = []
    for one in cluster_one:
        for two in cluster_two:
            if one in distance_matrix.columns and two in distance_matrix.columns:
                distances.append(distance_matrix[one][two])
    if linkage == 'average':
        return sum(distances) / (len(cluster_one)*len(cluster_two))
    if linkage == 'complete':
        return max(distances)
    if linkage == 'single':
        return min(distances)


def silhouette_score(distance_matrix, ids_cluster):
    cluster_sizes = ids_cluster['cluster_index'].value_counts().to_dict()
    # =============================================================================
    # calculate inter and intra distance for every entity
    # =============================================================================
    distances = {}
    for entity in ids_cluster[0]:
        current_cluster = ids_cluster[ids_cluster[0]==entity]['cluster_index'].iloc[0]
        distances[entity]={'intra_dist':intra_distance(list(ids_cluster[ids_cluster['cluster_index']==current_cluster][0]), distance_matrix, 'average', entity),
                          'inter_dist':min_inter_distance_to_entity(entity, distance_matrix, current_cluster, ids_cluster)}
    silhouette_score = 0
    intra_silhouette_scores = dict()
    for entity in distances:
        current_cluster = ids_cluster[ids_cluster[0]==entity]['cluster_index'].iloc[0]
        score = ((distances[entity]['inter_dist']-distances[entity]['intra_dist'])/
                             max(distances[entity]['inter_dist'],distances[entity]['intra_dist'])) if cluster_sizes[current_cluster] > 1 else 0
        # =============================================================================
        # save score for every cluster separately
        # =============================================================================
        if current_cluster not in intra_silhouette_scores:
            intra_silhouette_scores[current_cluster] = 0
        intra_silhouette_scores[current_cluster] += score
        # =============================================================================
        # save for total score
        # =============================================================================
        silhouette_score += score
    for cluster in cluster_sizes:
        intra_silhouette_scores[ids_cluster[ids_cluster['cluster_index']==cluster][1].iloc[0]] = intra_silhouette_scores[cluster] / cluster_sizes[cluster]
        del intra_silhouette_scores[cluster]
    return silhouette_score/len(distances), intra_silhouette_scores


def dunn_index(distance_matrix, ids_cluster):
    clusters = ids_cluster['cluster_index'].unique()
    max_intra_dist = None
    min_inter_dist = None
    for i in range(len(clusters)):
        distance = intra_distance(list(ids_cluster[ids_cluster['cluster_index'] == clusters[i]][0]),
                                  distance_matrix, 'average')
        if max_intra_dist is None or max_intra_dist < distance:
            max_intra_dist = distance

        for j in range(i + 1, len(clusters)):
            distance = inter_distance(ids_cluster[ids_cluster['cluster_index'] == clusters[i]][0],
                                      ids_cluster[ids_cluster['cluster_index'] == clusters[j]][0],
                                      distance_matrix, 'average')
            if min_inter_dist is None or min_inter_dist > distance:
                min_inter_dist = distance
    return min_inter_dist / max_intra_dist

In [325]:
def compare_id_clusters(disease_clusters, id_type):
    disease_clusters['cluster_index'] = disease_clusters.groupby(1).ngroup()
    disease_clusters_df = get_disease_mapping(disease_set=disease_clusters[0], id_type=id_type)
    result = list()
    for attribute in disease_clusters_df.columns[1:]:
        subset_df = disease_clusters_df[disease_clusters_df[attribute].str.len() > 0].reset_index(drop=True)
        subset_clusters = disease_clusters[disease_clusters[0].isin(subset_df[id_type])]
        missing_values = len(disease_clusters)-len(subset_df)
        print("Missing values for "+attribute+" :"+str(missing_values)+"/"+str(len(disease_clusters_df)))
        dist_mat = get_distance_matrix(subset_df[attribute])
        dist_df = pd.DataFrame(dist_mat, columns = subset_df[id_type], index = subset_df[id_type])
        ss_score = silhouette_score(distance_matrix = dist_df, ids_cluster = subset_clusters)
        di_score = dunn_index(distance_matrix = dist_df, ids_cluster = subset_clusters)
        result.append([attribute, di_score, ss_score[0], ss_score[1]])
    return result

In [292]:
subset_clusters = disease_clusters[disease_clusters[0].isin(subset_df['ICD-10'])]
subset_clusters

,0,1,2,cluster_index
0,G43,567,Migraine,11
2,E10,438,Type1 diabetes mellitus,9
3,E11,438,Type2 diabetes mellitus,9
4,J45,438,Asthma,9
5,E13,438,Other specified diabetes mellitus,9
6,E66,438,Obesity,9
7,G30,438,Alzheimer,9
8,I15,438,Secondary hypertension,9
9,I21,438,Acute myocardial infarction,9
10,I22,438,Subsequent myocardial imfarction,9


In [324]:
silhouette_score(distance_matrix = dist_df, ids_cluster = subset_clusters)

(-0.387602286947478,
 {438: -0.5990217161915569,
  567: 0.0,
  255: 0.0,
  239: 0.0,
  70: 0.0,
  24: 0.0,
  1: 0.0})

In [286]:
dunn_index(distance_matrix = dist_df, ids_cluster = subset_clusters)

0.7944838479005655

In [220]:
dist_df

ICD-10,E10,E11,E13,E14,E66,F00,F01,G20,G30,G43,I11,I12,I15,I21,I22,I50,I70,J45
ICD-10,,,,,,,,,,,,,,,,,,
E10,0.000000,0.158879,0.234375,0.000000,0.444210,0.577947,0.294931,0.435606,0.475909,0.480315,0.31250,0.439024,0.533333,0.392765,0.478889,0.512535,0.477842,0.451444
E11,0.158879,0.000000,0.828125,0.158879,0.364486,0.859813,0.934579,0.906542,0.514019,0.981308,0.93750,0.975610,0.476636,0.887850,0.607477,0.710280,0.635514,0.943925
E13,0.234375,0.828125,0.000000,0.234375,0.453125,0.765625,0.953125,0.812500,0.343750,0.953125,1.00000,1.000000,0.437500,0.859375,0.703125,0.531250,0.625000,0.968750
E14,0.000000,0.158879,0.234375,0.000000,0.442984,0.577947,0.294931,0.435606,0.474354,0.480315,0.31250,0.439024,0.533761,0.392765,0.478889,0.513649,0.477842,0.451444
E66,0.444210,0.364486,0.453125,0.442984,0.000000,0.534854,0.267281,0.446970,0.542135,0.409449,0.34375,0.292683,0.500855,0.403101,0.473889,0.484680,0.475434,0.448819
F00,0.577947,0.859813,0.765625,0.577947,0.534854,0.000000,0.691244,0.685606,0.394170,0.661417,0.75000,0.829268,0.564005,0.846253,0.667934,0.662864,0.676806,0.834646
F01,0.294931,0.934579,0.953125,0.294931,0.267281,0.691244,0.000000,0.801843,0.152074,0.834646,0.81250,0.878049,0.299539,0.571429,0.382488,0.400922,0.350230,0.774194
G20,0.435606,0.906542,0.812500,0.435606,0.446970,0.685606,0.801843,0.000000,0.337121,0.850394,0.90625,0.829268,0.492424,0.715909,0.541667,0.553030,0.541667,0.859848
G30,0.475909,0.514019,0.343750,0.474354,0.542135,0.394170,0.152074,0.337121,0.000000,0.362205,0.31250,0.341463,0.508120,0.369509,0.438333,0.478552,0.445568,0.459318


In [326]:
compare_id_clusters(disease_clusters=disease_clusters, id_type='ICD-10')

Missing values for disgenet.genes_related_to_disease :7/20
Missing values for disgenet.variants_related_to_disease :8/20
Missing values for ctd.pathway_related_to_disease :8/20


[['disgenet.genes_related_to_disease',
  1.098733357236396,
  -0.3010950528989328,
  {438: -0.5004120346138381,
   567: 0.0,
   255: 0.0,
   239: 0.0,
   171: 0.0,
   24: 0.042410714285714246}],
 ['disgenet.variants_related_to_disease',
  1.6783046854758819,
  -0.2148175149555411,
  {438: -0.3319907049312908,
   567: 0.0,
   255: 0.0,
   239: 0.0,
   171: 0.0,
   24: 0.0,
   1: 0.0}],
 ['ctd.pathway_related_to_disease',
  0.7944838479005655,
  -0.387602286947478,
  {438: -0.5990217161915569,
   567: 0.0,
   255: 0.0,
   239: 0.0,
   70: 0.0,
   24: 0.0,
   1: 0.0}]]